# Chapter 10: DPO (Direct Preference Optimization) Training

## 1. 학습 목표

* RLHF와 DPO의 차이점을 이해하고 DPO의 장점을 파악한다.
* 선호도 데이터셋(Prompt, Chosen, Rejected)을 구축하고 전처리한다.
* `DPOTrainer`를 사용하여 SFT 모델을 인간의 가치관에 맞게 정렬한다.
* QLoRA 환경에서 메모리 효율적으로 DPO를 수행하는 방법을 익힌다.

## 2. DPO란?

### 2.1 RLHF vs DPO

기존의 **RLHF (Reinforcement Learning from Human Feedback)** 방식은 보상 모델(Reward Model) 학습, PPO(Proximal Policy Optimization)를 통한 강화학습이라는 복잡한 단계를 거쳐야 했다. 이는 학습이 불안정하고 하이퍼파라미터 튜닝이 매우 어렵다는 단점이 있다.

반면 **DPO (Direct Preference Optimization)**는 별도의 보상 모델 없이, 선호 데이터(Chosen/Rejected)를 사용하여 언어 모델의 정책을 직접 최적화한다.

* **장점**: 구현이 단순하고, 학습이 안정적이며, RLHF와 동등하거나 더 나은 성능을 보인다.

### 2.2 DPO의 핵심 원리

DPO는 모델이 "비선호 응답(Rejected)"보다 "선호 응답(Chosen)"을 생성할 확률을 높이도록 학습한다.


DPO 손실 함수:
$$L_DPO = -log(σ(β * (log(π_policy(y_w|x)/π_ref(y_w|x)) - log(π_policy(yl|x)/π_ref(yl|x)))))$$

- $yw$: 선호 응답 (Chosen)
- $yl$: 비선호 응답 (Rejected)
- $π_policy$: 학습 중인 모델
- $π_ref$: 기준 모델 (SFT 완료된 모델, 동결됨)
- $β(Beta)$: 기준 모델에서 얼마나 벗어날지 조절하는 파라미터 (일반적으로 0.1)


## 3. 라이브러리 및 모델 준비

DPO 학습을 위해 필요한 라이브러리를 임포트하고, 이전 단계(Chapter 06~09)에서 SFT를 마친 모델을 로드한다고 가정한다. 실습에서는 `gpt-oss-20b` 를 사용한다.

> **중요**: DPO는 '학습 모델'과 '참조 모델(Reference Model)' 두 개가 필요하다. 하지만 PEFT(LoRA)를 사용하면, 베이스 모델을 참조 모델로 쓰고 어댑터만 학습하므로 메모리 사용량을 획기적으로 줄일 수 있다.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import DPOTrainer, DPOConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset, load_dataset

# SFT가 완료된 모델 경로 (또는 베이스 모델)
model_id = "openai/gpt-oss-20b"
#model_id = "Qwen/Qwen3-14B"

# 2. 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="eager" # Qwen3-14B의 경우 "sdpa"
)

# 3. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## 4. 선호도 데이터셋 준비

DPO 학습을 위해서는 세 가지 필드가 포함된 데이터셋이 필요하다.

1. **prompt**: 사용자 질문
2. **chosen**: 더 나은 답변 (선호)
3. **rejected**: 덜 좋은 답변 (비선호 - 환각, 편향, 불친절 등)

In [3]:
def prepare_preference_dataset():
    """
    DPO 학습을 위한 선호도 데이터셋을 생성하거나 로드하는 함수다.
    """
    # 예시 데이터 (실제로는 Anthropic/hh-rlhf 등을 사용)
    data = [
        {
            "prompt": "재귀(Recursion)를 초보자에게 설명해줘.",
            "chosen": "재귀는 러시아 인형(마트료시카)과 비슷합니다. 인형을 열면 더 작은 인형이 나오고, 가장 작은 인형이 나올 때까지 계속되는 구조와 같습니다. 프로그래밍에서는 함수가 자기 자신을 호출하는 것을 의미합니다.",
            "rejected": "재귀는 함수가 자기 자신을 호출하는 프로그래밍 기법이다. 종료 조건이 없으면 스택 오버플로우가 발생한다."
            # Rejected 이유: 설명이 너무 딱딱하고 초보자에게 불친절함
        },
        {
            "prompt": "Python의 장점을 설명해줘.",
            "chosen": "Python은 문법이 간결하여 읽기 쉽고, 데이터 과학 및 AI 분야의 풍부한 라이브러리를 보유하고 있습니다. 또한 커뮤니티가 매우 활발합니다.",
            "rejected": "Python은 좋은 언어이다."
            # Rejected 이유: 너무 단답형이고 정보가 부족함
        }
    ] * 50 # 실습을 위해 데이터 복제

    dataset = Dataset.from_list(data)
    return dataset

train_dataset = prepare_preference_dataset()
print(f"학습 데이터 개수: {len(train_dataset)}")
print(f"데이터 예시: {train_dataset[0]}")

학습 데이터 개수: 100
데이터 예시: {'prompt': '재귀(Recursion)를 초보자에게 설명해줘.', 'chosen': '재귀는 러시아 인형(마트료시카)과 비슷합니다. 인형을 열면 더 작은 인형이 나오고, 가장 작은 인형이 나올 때까지 계속되는 구조와 같습니다. 프로그래밍에서는 함수가 자기 자신을 호출하는 것을 의미합니다.', 'rejected': '재귀는 함수가 자기 자신을 호출하는 프로그래밍 기법이다. 종료 조건이 없으면 스택 오버플로우가 발생한다.'}


## 5. DPO 학습 설정

`DPOTrainer`를 설정한다. 여기서 가장 중요한 하이퍼파라미터는 `beta`이다. `beta`가 클수록 참조 모델(기존 모델)에서 덜 벗어나려고 하며, 작을수록 선호 데이터에 더 과감하게 맞춰진다. 보통 0.1을 사용한다.

In [4]:
# LoRA 설정
# DPO는 SFT보다 과적합되기 쉬우므로 Rank를 낮추거나 Dropout을 신경 써야 한다.
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# DPO 설정
training_args = DPOConfig(
    output_dir="./GPT-OSS-20B-DPO",

    # 핵심 파라미터
    beta=0.1,                        # DPO 온도 파라미터 (0.1 ~ 0.5 권장)

    # 학습 파라미터
    num_train_epochs=1,
    per_device_train_batch_size=2,   # DPO는 메모리를 더 많이 사용하므로 배치 크기를 줄임
    gradient_accumulation_steps=8,
    learning_rate=5e-6,              # SFT보다 훨씬 낮은 학습률 사용 (중요!)

    # 최적화
    optim="paged_adamw_8bit",
    fp16=False,
    bf16=True,

    # 데이터 처리
    max_length=1024,                 # 프롬프트 + 응답 최대 길이
    max_prompt_length=512,           # 프롬프트 최대 길이
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    report_to="none"
)

# Trainer 생성
# DPOTrainer는 내부적으로 모델을 복제하여 Reference Model로 사용하지만,
# peft_config를 제공하면 LoRA 어댑터만 학습하므로 메모리 효율적이다.
trainer = DPOTrainer(
    model=model,
    ref_model=None,             # PEFT 사용 시 None으로 설정 (자동 처리)
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer, # 토크나이저 전달
    peft_config=peft_config,    # LoRA 설정 전달
)

print("DPO Trainer 설정 완료")

Extracting prompt in train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

DPO Trainer 설정 완료


## 6. 학습 실행 및 저장

DPO 학습을 실행한다. 학습 과정에서 `rewards/chosen`과 `rewards/rejected` 로그를 확인하여, 선호 응답의 보상 점수가 올라가는지 모니터링해야 한다.

In [5]:
print("DPO 학습 시작...")
trainer.train()
print("학습 완료!")

# 모델 저장
trainer.save_model("./GPT-OSS-20B-DPO-Final")
print("모델 저장 완료: ./GPT-OSS-20B-DPO-Final")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 199998}.


DPO 학습 시작...


/home/josh/anaconda3/envs/llm/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss


학습 완료!
모델 저장 완료: ./GPT-OSS-20B-DPO-final


## 7. 요약

이 챕터에서는 **DPO**를 사용하여 모델을 인간의 선호도에 정렬시키는 방법을 학습했다.

1. **DPO의 효율성**: 보상 모델 없이 선호 데이터만으로 최적화가 가능하다.
2. **데이터셋 구조**: (Prompt, Chosen, Rejected) 쌍이 필요하다.
3. **학습 주의점**: SFT보다 훨씬 낮은 학습률(5e-6 등)을 사용해야 하며, `beta` 파라미터로 변화 폭을 조절한다.
4. **메모리 관리**: PEFT를 활용하면 Reference Model을 별도로 로드하지 않아도 되어 20B 모델도 학습 가능하다.

이것으로 기본적인 정렬(Alignment) 과정까지 마쳤다. 다음 챕터에서는 단순한 선호도를 넘어, 복잡한 논리적 추론 능력을 강화하는 최신 기법인 **GRPO**에 대해 다룬다.

다음 챕터는 **Chapter 11: GRPO를 활용한 추론 모델 개발**이다.